In [74]:
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import numpy as np
import re
import time


In [75]:
from fugue_notebook import setup
setup()

<IPython.core.display.Javascript object>

## Thu thập dữ liệu 

In [76]:
ma_so_du_thi = []
with open('./ma_so_thi_sinh.txt', 'r') as f:
    msdt = f.readlines()
for ma_so in msdt:
    ma_so_du_thi.append(ma_so.rstrip('\n'))
len(ma_so_du_thi)

6912

In [77]:
url = 'https://vtv.vn/tra-cuu-diem-thi-2023.htm'
path_input = '//div/div[@class="textsearchall"]/input'

In [78]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)

In [59]:
diem_thi = []
input_sbd = driver.find_element(by=By.XPATH, value=path_input)
btn_tim_kiem = driver.find_element(by=By.ID, value='btnSearchDiemThi')
for sbd in ma_so_du_thi:
    input_sbd.send_keys(sbd)
    btn_tim_kiem.click()
    ket_qua = driver.find_elements(by=By.ID, value = 'searchresult')
    for item in ket_qua:
        diem_thi.append(item.text.split('\n'))
    time.sleep(1)
    input_sbd.clear()


In [79]:
chu_thich_ten_ngoai_ngu = driver.find_elements(by=By.CLASS_NAME, value='chu-thich')
for ngoai_ngu in chu_thich_ten_ngoai_ngu:
    p_chu_thich = ngoai_ngu.text.split(': ')
ngoai_ngu = p_chu_thich[1:]
dic_ngoai_ngu = {}
for ngng in ngoai_ngu:
    for language in ngng.split(','):
        dic_ngoai_ngu[language.split(' - ')[0]] = language.split(' - ')[1]
dic_ngoai_ngu

{'N1': 'Tiếng Anh',
 ' N2': 'Tiếng Nga',
 ' N3': 'Tiếng Pháp',
 ' N4': 'Tiếng Trung',
 ' N5': 'Tiếng Đức',
 ' N6': 'Tiếng Nhật'}

In [80]:
driver.close()

## Xử lý và phân tích dữ liệu 

In [62]:
col_ = []
col_name = []
for item in diem_thi[1]:
    col_.append(item.split(':')[0])
for item in col_:
    x = item.rstrip(' ')
    col_name.append(x)

In [63]:
df_diem_thi_hau_giang = pd.DataFrame(diem_thi[1:], columns = col_name )
df_diem_thi_hau_giang.head(3)

,Số báo danh,Toán,Văn,Lý,Hóa,Sinh,Sử,Địa,Giáo dục công dân,Ngoại ngữ,Mã môn ngoại ngữ
0,Số báo danh :64000001,Toán :6.8,Văn :8,Lý :6.5,Hóa :7,Sinh :6.25,Sử :,Địa :,Giáo dục công dân :,Ngoại ngữ :7,Mã môn ngoại ngữ :N1
1,Số báo danh :64000002,Toán :2.4,Văn :5,Lý :,Hóa :,Sinh :,Sử :4.5,Địa :5.25,Giáo dục công dân :,Ngoại ngữ :,Mã môn ngoại ngữ :null
2,Số báo danh :64000003,Toán :7.4,Văn :7.75,Lý :,Hóa :,Sinh :,Sử :8.25,Địa :7,Giáo dục công dân :9.5,Ngoại ngữ :6.4,Mã môn ngoại ngữ :N1


In [64]:
for col in df_diem_thi_hau_giang.columns[:-1]:
    df_diem_thi_hau_giang[col] = \
        df_diem_thi_hau_giang[col].str.extract('(\d+.?\d+)')
df_diem_thi_hau_giang.iloc[:,-1] = df_diem_thi_hau_giang.iloc[:,-1].str.extract('(\D{1}\d$|\D{4}$)')

In [65]:
df_diem_thi_hau_giang.head()

,Số báo danh,Toán,Văn,Lý,Hóa,Sinh,Sử,Địa,Giáo dục công dân,Ngoại ngữ,Mã môn ngoại ngữ
0,64000001,6.8,NaN,6.5,NaN,6.25,NaN,NaN,NaN,NaN,N1
1,64000002,2.4,NaN,NaN,NaN,NaN,4.5,5.25,NaN,NaN,null
2,64000003,7.4,7.75,NaN,NaN,NaN,8.25,NaN,9.5,6.4,N1
3,64000004,5.6,6.5,NaN,NaN,NaN,7.5,5.25,8.5,NaN,N1
4,64000005,5.2,6.25,5.75,5.5,NaN,NaN,NaN,NaN,NaN,N1


In [66]:
df_diem_thi_hau_giang['Tên ngoại ngữ'] = df_diem_thi_hau_giang.iloc[:,-1].map(dic_ngoai_ngu)

## Xuất tập tin ra csv

In [67]:
df_diem_thi_hau_giang.drop_duplicates(inplace=True)
df_diem_thi_hau_giang.to_csv('diem_thi_THPT_HG.csv')

In [68]:
df_diem_thi_hau_giang.head()

,Số báo danh,Toán,Văn,Lý,Hóa,Sinh,Sử,Địa,Giáo dục công dân,Ngoại ngữ,Mã môn ngoại ngữ,Tên ngoại ngữ
0,64000001,6.8,NaN,6.5,NaN,6.25,NaN,NaN,NaN,NaN,N1,Tiếng Anh
1,64000002,2.4,NaN,NaN,NaN,NaN,4.5,5.25,NaN,NaN,null,NaN
2,64000003,7.4,7.75,NaN,NaN,NaN,8.25,NaN,9.5,6.4,N1,Tiếng Anh
3,64000004,5.6,6.5,NaN,NaN,NaN,7.5,5.25,8.5,NaN,N1,Tiếng Anh
4,64000005,5.2,6.25,5.75,5.5,NaN,NaN,NaN,NaN,NaN,N1,Tiếng Anh


In [69]:
df_ = df_diem_thi_hau_giang.iloc[:,1:]
df_.describe()

,Toán,Văn,Lý,Hóa,Sinh,Sử,Địa,Giáo dục công dân,Ngoại ngữ,Mã môn ngoại ngữ,Tên ngoại ngữ
count,5307,4800,1463,1445,1482,3687,3716,3359,5010,6879,6303
unique,34,48,24,25,24,27,24,19,37,4,1
top,5.8,6.75,6.75,7.5,6.5,5.75,6.25,8.5,3.6,N1,Tiếng Anh
freq,315,661,136,169,166,302,459,432,396,6303,6303


In [70]:
df_diem_thi_hau_giang = pd.read_csv('diem_thi_THPT_HG.csv', index_col = False)
df_diem_thi_hau_giang = df_diem_thi_hau_giang.iloc[:,1:]

In [71]:
df_diem_thi_hau_giang.reset_index(drop=True)
diem_thi=df_diem_thi_hau_giang.iloc[:,1:-2]
diem_thi

,Số báo danh,Toán,Văn,Lý,Hóa,Sinh,Sử,Địa,Giáo dục công dân,Ngoại ngữ,Mã môn ngoại ngữ,Tên ngoại ngữ
0,64000001,6.8,NaN,6.50,NaN,6.25,NaN,NaN,NaN,NaN,N1,Tiếng Anh
1,64000002,2.4,NaN,NaN,NaN,NaN,4.50,5.25,NaN,NaN,NaN,NaN
2,64000003,7.4,7.75,NaN,NaN,NaN,8.25,NaN,9.50,6.4,N1,Tiếng Anh
3,64000004,5.6,6.50,NaN,NaN,NaN,7.50,5.25,8.50,NaN,N1,Tiếng Anh
4,64000005,5.2,6.25,5.75,5.50,NaN,NaN,NaN,NaN,NaN,N1,Tiếng Anh
...,...,...,...,...,...,...,...,...,...,...,...,...
6874,64006932,6.6,7.75,5.75,NaN,8.25,NaN,NaN,NaN,NaN,N1,Tiếng Anh
6875,64006933,7.8,6.75,NaN,NaN,NaN,8.50,7.75,9.75,5.4,N1,Tiếng Anh
6876,64006934,7.4,7.50,NaN,5.75,6.25,NaN,NaN,NaN,NaN,N1,Tiếng Anh
6877,64006935,6.4,NaN,NaN,NaN,NaN,5.50,5.75,7.25,NaN,N1,Tiếng Anh


## Thống kê số lượng điểm thi của từng môn

In [73]:
%%fsql
SELECT COUNT(`Toán`) AS SL_Toan, COUNT(`Văn`) AS SL_Van, COUNT(`Lý`) AS SL_Ly,
       COUNT(`Hóa`) AS SL_Hoa, COUNT(`Sinh`) AS SL_Sinh, COUNT(`Sử`) AS SL_Su,
       COUNT(`Địa`) AS SL_Dia, COUNT(`Giáo dục công dân`) AS SL_GDCD, COUNT(`Ngoại ngữ`) AS SL_NgoaiNgu
FROM diem_thi
PRINT 

,SL_Toan:long,SL_Van:long,SL_Ly:long,SL_Hoa:long,SL_Sinh:long,SL_Su:long,SL_Dia:long,SL_GDCD:long,SL_NgoaiNgu:long
0,5307,4800,1463,1445,1482,3687,3716,3359,5010
